In [52]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks

In [46]:
# Define helper functions for statistical metrics
def compute_metrics(signal):
    mean = np.mean(signal)
    zero_crossing_rate = ((signal[:-1] * signal[1:]) < 0).sum() / len(signal)
    signal_range = np.ptp(signal)
    energy = np.sum(np.square(signal))
    rms = np.sqrt(np.mean(np.square(signal)))
    variance = np.var(signal)
    
    return {
        'Mean': mean,
        'Zero Crossing Rate': zero_crossing_rate,
        'Range': signal_range,
        'Energy': energy,
        'RMS': rms,
        'Variance': variance
    }

In [49]:
# Plot individual channels and superimposed signals
def plot_signals(data, class_name, sample_id):
    n_channels = data.shape[0]  # Initialize number of channels from data shape
    time = np.arange(data.shape[1])  # Initialize time based on data length
    # Create folder for saving plots
    output_folder = f"plots/class_{class_name}_sample_{sample_id}"
    os.makedirs(output_folder, exist_ok=True)
    
    combined_signal = np.zeros_like(data[0])
    for i in range(n_channels):
        plt.figure()
        plt.plot(time, data[i], label=f"Channel {i+1}")
        plt.xlabel("Time")
        plt.ylabel("Amplitude")
        plt.title(f"EEG Signal for Channel {i+1} (Class: {class_name}, Sample: {sample_id})")
        plt.legend()
        plt.savefig(f"{output_folder}/channel_{i+1}.png")
        plt.close()
        combined_signal += data[i]

    # Plot combined signal
    plt.figure()
    for i in range(n_channels):
        plt.plot(time, data[i], label=f"Channel {i+1}")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.title(f"Superimposed EEG Signals (Class: {class_name}, Sample: {sample_id})")
    plt.legend()
    plt.savefig(f"{output_folder}/combined_signal.png")
    plt.close()


    

In [50]:
 #Main function to process and visualize data

def process_data(data_path, classes):
    all_metrics = []

    for class_name in classes:
        class_folder = os.path.join(data_path, class_name)
        samples = os.listdir(class_folder)[:1]  # Select the first sample for demonstration

        for sample_id, sample_file in enumerate(samples):
            sample_path = os.path.join(class_folder, sample_file)
            data = np.load(sample_path)  # Assuming data is saved as .npy file

            # Plot signals
            plot_signals(data, class_name, sample_id)

            # Compute metrics
            for i in range(data.shape[0]):
                metrics = compute_metrics(data[i])
                metrics['Class'] = class_name
                metrics['Sample'] = sample_id
                metrics['Channel'] = f"Channel {i+1}"
                all_metrics.append(metrics)

    return all_metrics

In [51]:
# Path to the data folder and class names
data_path = r"C:/Users/User/Desktop/Hackathon/Impulse/train_data"  # Update this path
classes = ["Complex_Partial_Seizures", "Electrographic_Seizures", "Normal", "Video_detected_Seizures_with_no_visual_change_over_EEG"]

# Process data and retrieve metrics
metrics = process_data(data_path, classes)

# Save metrics to CSV and Excel
metrics_df = pd.DataFrame(metrics)
metrics_csv_path = os.path.join("output", "eeg_metrics.csv")
metrics_excel_path = os.path.join("output", "eeg_metrics.xlsx")
os.makedirs("output", exist_ok=True)
metrics_df.to_csv(metrics_csv_path, index=False)
metrics_df.to_excel(metrics_excel_path, index=False)

print(f"Metrics saved to {metrics_csv_path} and {metrics_excel_path}")


Metrics saved to output\eeg_metrics.csv and output\eeg_metrics.xlsx
